## Исследование продуктов по автодополнению кода

*Обзор получился хейтерским, весь текст ниже это субъективное мнение и возможно я не прав, буду рад ошибаться*

---

На рынке по автодополнению кода сейчас есть множество вариантов. При этом есть как внешние продукты, которые интегрируются независимо от языка и среды разработки, например, [```GitHub Copilot```](https://github.com/features/copilot), так и встроенные в конкретную среду разработки, например, [```IntelliSense```](https://code.visualstudio.com/docs/editor/intellisense) для ```VisualStudio```. Но насколько они разные и качественные?

Из наиболее популярных сразу же можно назвать [```Tabnine```](https://www.tabnine.com), который находится на слуху из-за агрессивной рекламной политики и по ощущениям больше ориентирован в сторону ```JavaScript```, коммьюнити которого одно из самых больших. Отсюда и популярность.

---

## Tabnine

<img src="images/tabnine.jpg" width="400">

---

[```Tabnine```](https://www.tabnine.com) предсказывает и предлагает варианты продолжения кода, основываясь на контексте текущего кода, прошлого опыта и синтаксиса языка. Может быть внедрен в большое количество сред разработки, на официальном сайте указаны следующие:
<img src="images/tabnine_ide.jpg" width="600">

Поддерживаются многие языки программирования и популярные фреймворки:<br>
<img src="images/tabnine_frameworks.jpg" width="600">

По умолчанию устанавливается стартовый тариф, который предлагает базовый функционал и иcпользует модель, обученную на открытом исходном коде с разрешительными лицензиями. Такая опция не предусматривает никакой оптимизации под собственные нужды, то есть нельзя учитывать опыт написания своего кода. Для этого нужно купить подписку и перейти на следующий уровень.




Для проекта есть свой [```репозиторий```](https://github.com/codota/TabNine) на гитхабе, но исходного кода там нет. Таким образом, во-первых, непонятно какая архитектура лежит в основе модели, а во-вторых, это коммерческое программное обеспечение, и могут возникать сомнения насчет приватности, однако на сайте заявлено что приватность гарантируется. Также на сайте удалось найти краткое описание того как ```Tabnine``` устроен внутри:

Внутри лежит обученная ```LLM``` модель, основанная на трансформере, которая потребялет упорядоченные данные и генерирует ответы. Попробую предположить что просто берется декодерная часть обученного трансформера, что-то типа ```GPT```.

Процесс выглядит примерно так ([```источник```](https://support.tabnine.com/hc/en-us/articles/5407860396817-How-Tabnine-works)):

1. При написании кода, символы с обеих сторон курсора, если они существуют, токенизируются и шифруются.
2. Затем зашифрованные пакеты токенов отправляются на инференс сервер, на котором лежит обученная модель.
3. Инференс сервер дешифрует пакеты, передает токены в модель и генерирует наиболее вероятное продолжение.
4. Найденное продолжение шифруется и отправляется обратно клиенту, где происходит расшифровка и подстановка.

То есть безусловно будут издержки на отправку пакетов по интернету и вычислительные ресурсы машины. Имея не особо мощные ресурсы, я заметил небольшое подтормаживание при использовании ```Tabnine```, что в целом логично.

Теперь подробнее рассмотрим основной функционал. На сайте заявлены следующие возможности:

* Продолжение кода всей строки
* Написание функций и классов по сигнатуре
* Генерация кода через документацию или комментарии

Из последнего следует, что модель бимодальная, и была обучена в частности на парах с кодом и естественным описанием. Продолжение кода не выглядит как в лекции у ```Grammformer```, то есть нет пропущенных токенов, возможно просто код который был написан не был особо сложным.

### Примеры использования

Без труда получилось сгенерировать функцию чтения из файла. Это часто используемое действие, поэтому логично что в обучающей выборке такой сценарий встречался. В предложенном варианте читается текстовый файл, вообще на выбор было несколько, но они одинаковые с точностью до кавычек и названий переменных. Если попросить сгенерировать чтение бинарных данных, то проблем также не возникнет.

<img src="images/tabnine_read_file.jpg" width="600">

Хорошо, это было просто и вполне ожидаемо. Модель думает шаблонами, которые когда-то видела. Следующая задача чуть сложнее, но в целом тоже используется шаблон. Класс пользователя был написан вручную и дальше захотелось получить с командной строки параметры:

<img src="images/tabnine_user.jpg" width="600">

С простыми задачами проблем нет и в целом не должно быть. Это такие задачи, где неглубокий контекст решения или задача достаточно популярная и как следствие в обучающей выборке что-то такое было. 

Теперь попробую применить генерацию в существующем проекте, в простом приближении идея такая, что есть слушающий сокет, который по ```TCP``` принимает команды в виде байтов, эти байты парсятся в строчку, по строчке определяется команда. В коде есть классы, которые имплементируют интерфейс команды, сможет ли ```Tabnine``` по началу кода класса и параметрам дописать реализацию команды?

<img src="images/tabnine_child.jpg" width="600">

Попросил сделать реализацию индентичной команды, но только чтобы адрес выбирался рандомно. Шапка класса была предложена корректно, но нужный метод пустой, при этом альтернативных вариантов не было. Пришлось спуститься внутрь метода и еще раз написать хотелку 

<img src="images/tabnine_execute.jpg" width="600">

В целом да, но где тут рандом? Предлагается просто нулевой элемент. Окей, учитывая контекст ```Tabnine``` что-то может предложить, но если это не пара кликов и я не могу этому доверять, то зачем тратить время? Что-то дописывать руками и получать все равно не совсем то что хотел, ну такое.

Оценить все преимущества ```Tabnine``` не получится, потому что персонализация доступна только на платных тарифах. Кажется, что если модель будет не просто смотреть, но и учиться по собственному проекту, то шансы получить качественный код намного выше, но чтобы это проверить нужна подписка и время. Судя по отзывам кто-то действительно получает от этого профит. 

Но если честно, для меня туманно выглядят перспективы автодополнения в таком виде, в котором они сейчас есть. Сейчас это выезжает на вау эффекте, а модели то просто кидаются шаблонами, тупо запоминая контекст. К ожиданиям и субъективной оценке вернусь в конце.

В абзаце выше я не имел ввиду решения по типу ```IntelliSense``` от ```Microsoft```, которые хорошо справляются с задачами автодополнения, такие решения достаточно старые и впервые датируются аж прошлым веком, например ```Visual Basic 5.0 Control Creation Edition```. Они не берут на себя большую ответственность и не кидаются большими кусками кода, вместо этого аккуратно предлагают имена методов и атрибутов объектов. Довольно долго этим пользовался когда писал на шарпе.

---


## Copilot

<img src="images/copilot.jpg" width="400">

---

Следующий на очереди [```GitHub Copilot```](https://github.com/features/copilot). Для него нет бесплатной подписки, чтобы активировать пробный период нужно ввести платежные реквизиты. Это продукт от ```OpenAI```, который использует модель для генерации кода ```Codex```. На сайте не нашел ничего такого, чего не было бы в ```Tabnine```. Не думаю что это решение сильно удивило бы, потому как ```Codex``` это тоже просто частичка трансформера. Здесь я приведу мнение

<img src="images/lecun.jpg" width="1000">

Я с этим полностью согласен. Вообще тут можно немного ответвиться и поговорить об хайпе на такой подход: 
> А давайте возьмем ```BERT``` или ```GPT```, что-нибудь подкрутим там и оно само все решится

Мое мнение. Если модель популярее и больше, это не значит что она лучше. Нужно пытаться подходить к новой задаче основательнее и искать более естественные решения, уделять внимание в первую очередь данным, они важнее алгоритмов. Трансформеры это безусловно прорыв, но наверняка не истина в последней инстанции. Расточительно закидывать все в и без того огромные модели, придерживаясь стратегии "ну посмотрим что будет, может получится". А может и да, но какой ценой. Тем более когда получается решение, которому трудно доверять даже наполовину. Это имеет смысл в крупных компаниях, для которых прирост даже на доли процента может принести прибыль.

### Проблема среднего кода

В процессе исследования нашел интересное мнение на хабре. [```Проблема среднего кода```](https://habr.com/ru/company/haulmont/blog/568714/)

> Вообще, складывается такое впечатление, что модели учатся хорошему и плохому одновременно, не отдавая предпочтения ни тому, ни другому. Чему научились, то и делают. Давайте теперь задумаемся над простым вопросом: какого кода на ```GitHub``` больше, старого или нового? Хорошего или плохого? Вот в том-то и дело. <br><br>
Инструменты типа ```Copilot``` репродуцируют “средненький” код. А потом этот код летит на ```GitHub``` увеличивая выборку “средненького” кода. На котором снова учится модель. Получается сваливание к устойчивому положению равновесия или скатывание в потенциальную яму. Слово “средний” я применяю не характеризуя качество кода, а закладываю в него статистический смысл. Если у вас есть впечатление, что среднестатистический код на ```GitHub``` хорош, то волноваться незачем.

Дообучая модель, стремясь все к большей и большей выборке, появляется риск придти к “среднему” решению, напоминает закон больших чисел, где в качестве экспериментов выступают дообучения на все бóльшей выборке. Может быть в этом и нет ничего плохого.

---

## Kite

<img src="images/kite.jpg" width="400">

---

Дальше по списку не пойду, хотя есть еще десяток решений, в том числе от известных компаний типа ```Amazon``` ```CodeWhisperer``` и ```Microsoft``` ```IntelliCode``` (не путать с ```IntelliSense```), но если самые хайповые в таком виде, то ситуация примерно понятна.

Интереснее посмотреть на историю [```Kite```](https://www.kite.com/blog/product/kite-is-saying-farewell/). Это стартап на рынке по автодополнению кода, который недавно прекратил свое существование. Они одними из первых вышли на  рынок и предлагали в целом похожие услуги на те, что сейчас есть у ```Tabnine``` и ```GitHub Copilot```. В чем же причина падения? На официальном сайте довольно подробно описано [```из-за чего так получилось```](https://www.kite.com/blog/product/kite-is-saying-farewell/) по мнению руководителя стартапа, здесь же сделаю очень краткую выжимку, мне кажется это показательным и сценарий вполне может зациклиться.

> First, we failed to deliver our vision of AI-assisted programming because we were 10+ years too early to market, i.e. the tech is not ready yet.

> А что если сейчас еще рано? Сейчас определенно лучше чем десять лет назад, но все относительно. Что будет через десять лет? Судя по тем решениям что есть, до высокого качества еще далеко и в будущем может быть существенный прорыв, который обесценит те подходы, что есть сейчас, и тогда нынешние решения ждет та же участь, что и ```Kite```.

> You can see this in Github Copilot, which is built by Github in collaboration with Open AI. As of late 2022, Copilot shows a lot of promise but still has a long way to go. The largest issue is that state-of-the-art models don’t understand the structure of code, such as non-local context. We made some progress towards better models for code, but the problem is very engineering intensive. It may cost over $100 million to build a production-quality tool capable of synthesizing code reliably, and nobody has tried that quite yet.

> Большой плюс про непонимание структуры кода, нелокального контекста. Я думаю, что в этой задаче недостаточно просто увидеть зависимость или запомнить шаблоны, нужен какой-то творческий потенциал

> Then we grew our user base. We executed very well here, and grew our user base to 500,000 monthly-active developers, with almost zero marketing spend. Then, our product failed to generate revenue. Our 500k developers would not pay to use it.

> Готовы ли отдельные пользователи платить? Скорее всего нет, потому что нужна большая кодовая база, а это больше относится к отдельным командам в крупных команиях, которые пишут в продакшн, но при том качестве что есть сейчас стоит ли платить? Сократит ли это время на разработку при прочих равных условиях?

### Сократит ли это время на разработку при прочих равных условиях?

Это интересный вопрос, я нашел [```исследование```](https://github.blog/2022-09-07-research-quantifying-github-copilots-impact-on-developer-productivity-and-happiness/) на эту тему, когда на сайте ```Copilot``` увидел странное: "96% are faster with repetitive tasks"

Также можно посмотреть более научный вариант того же исследования [```Productivity assessment of neural code completion```](https://dl.acm.org/doi/10.1145/3520312.3534864)

--- 
## Итог

Возможно я отношусь к этой теме слишком скептически, и вероятно это связано с недостатком опыта, но если я получу новые наблюдения или другой положительный опыт, то без сомнения скорректирую свое мнение, у меня нет задачи отрицать прогресс. Я верю что он возможен.